In [6]:
import csv
import re
import spacy
from spacy import displacy

from owlready2 import *
import rdflib

import re

# pre-processing
def Filter(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")
            
# stopwords from parsing the whole sentence
def RemoveStopword1(phrase, doc, chunkStart, chunkEnd, stopList):
    result = phrase
    i_stop=0
    #start = chunk.start# to eliminate the condition when the first word of chunk is stop word
    for i_sen in range(chunkStart, chunkEnd):
        while i_stop < len(stopList) and stopList[i_stop] < i_sen-1:
            #print(str(stopList[i_stop]) + ' ' + str(i_sen))
            i_stop = i_stop+1
        # there is no stop word in current chunk
        if i_stop >= len(stopList):
            break;
        #print(i_sen)
        # finish going through the chunk
        if stopList[i_stop] > chunkEnd-1:
            break
        # find the stop word and remove it
        if stopList[i_stop] == i_sen-1:
            #print(doc[i_sen-1])
            if i_sen-1 == chunkStart:
                result = result.replace(doc[i_sen-1].text + ' ', '')
                chunkStart = chunkStart+1
            else:
                result = result.replace(' ' + doc[i_sen-1].text, '')
    return result

# stopwords from parsing triple separately
def RemoveStopword2(inputPhrase):
    result = ''
    doc_phrase = nlp(str(inputPhrase))
    for token in doc_phrase:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #       token.shape_, token.is_alpha, token.is_stop)
        if not token.is_stop:
            result = result + token.text + ' '
        #else:
        #    print(token.text + ', ', end = '')    
    return result


# extract one triple from given sentence
def ExtractTriple(sen):
    # initialize the triple and stop word list
    subj = ""
    pred = ""
    obj = ""
    stopList = []
    
    # parse sentence
    doc = nlp(str(sen))
    print('\n' + str(index) + ': ' + senSet[index])
    
    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')

    print('stop words: ', end='')
    for token in doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)

        # record the index of stop words
        if token.is_stop:
            print(token.text + ', ', end='')
            stopList.append(token.i)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.text
            '''#an earlier solution that I find not necessary
            obj = token.lemma_
            # to avoid cases like "-PRON-"
            if obj[0] == '-':
                obj = token.text'''
    print('\n')

    subj_1 = subj
    obj_1 = obj
    # using chunk to update subject and object
    for chunk in doc.noun_chunks:
        if chunk.root.head.text == pred_orig and re.match('nsubj', chunk.root.dep_):
            subj = chunk.text
            # remove stop words
            subj_1 = RemoveStopword1(subj, doc, chunk.start, chunk.end, stopList)

        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text
            # remove stop words
            obj_1 = RemoveStopword1(obj, doc, chunk.start, chunk.end, stopList)
        #print(chunk.text + ' ' + str(chunk.start))
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)

    print('Before : ' + subj + ' - ' + pred + ' - ' + obj)
    print('Method1: ' + subj_1 + ' - ' + pred + ' - ' + obj_1)

    # second method to remove stop words
    subj_2 = RemoveStopword2(subj)
    obj_2 = RemoveStopword2(obj)
    print('Method2: ' + subj_2 + '- ' + pred + ' - ' + obj_2 + '\n')

    return [subj, pred, obj]

# transfer a phrase to a URI form
def FormatURI(phrase):
    #print('Before formatting:  ' + phrase)
    chars = list(phrase)
    if len(chars) > 0:
        chars[0] = chars[0].upper()
    for i in range(len(chars)):
        if chars[i] == ' ' and i+1 < len(chars):
            chars[i+1] = chars[i+1].upper()
    phrase = ''.join(chars)
    phrase = phrase.replace(' ', '')
    phrase = re.sub(r'[^a-zA-Z0-9\s]', '', phrase)
    #print('After formatting:  ' + phrase)
    return phrase

# query the given triple in the ontology with SPARQL
# return true/false as result
def QueryTriple(subj, pred, obj):
    prefix = """
    PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpd:<http://dbpedia.org/ontology/>
    """
    #subj = "provinceLink"
    #pred = "range"
    #obj = "Province"
    qSelect = prefix + """
    SELECT ?sub WHERE {
      ?sub rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    qAsk = prefix + """
    ASK {
        dbpd:""" + FormatURI(subj) + """ rdf:""" + FormatURI(pred) + """ dbpd:""" + FormatURI(obj) + """.
    }"""
    
    r = list(m_graph.query(qAsk))
    return r

# load Spacy NLP dictionary
nlp = spacy.load('en_core_web_sm')

# load DBPD ontology and construct graph for query
m_world = World()# Owlready2 stores every triples in a ‘World’ object
m_onto = m_world.get_ontology("dbpedia.owl").load()
m_graph = m_world.as_rdflib_graph()

# load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)
senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array
file.close()
print("Total sentences: " + str(len(senSet)))

# pre-processing
Filter(senSet)

# parse and query each sentence
for index in range(len(senSet)):
    #index = 13
    
    # extract triple from current sentence
    [subj, pred, obj] = ExtractTriple(senSet[index])
    print('Triple for Query   : ' + subj + ' - ' + pred + ' - ' + obj)

    # query the triple in dbpd with SPARQL
    queryResult = QueryTriple(subj, pred, obj)
    print('Triple Query Result: ' + str(queryResult))
    
    

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataPrope

Total sentences: 534
Pre-processing...

0: Yeah full detail with everyone here
stop words: full, with, everyone, here, 

Before :  - detail - 
Method1:  - detail - 
Method2: - detail - 

Triple for Query   :  - detail - 
Triple Query Result: [False]

1:   So um you guys I'll just do introductions again
stop words: you, just, do, again, 

Before : I - do - introductions
Method1: I - do - introductions
Method2: I - do - introductions 

Triple for Query   : I - do - introductions
Triple Query Result: [False]

2:   Um you guys have met me
stop words: you, have, me, 

Before : you guys - meet - me
Method1: guys - meet - me
Method2: guys - meet - 

Triple for Query   : you guys - meet - me
Triple Query Result: [False]

3:   I'm Tyler
stop words: 

Before :   I - be - Tyler
Method1:   I - be - Tyler
Method2:    I - be - Tyler 

Triple for Query   :   I - be - Tyler
Triple Query Result: [False]

4:   Um I have worked here and I do the recruits department
stop words: have, here, and, do, the, 


Method2: I - see - wheelchairs 

Triple for Query   : I - see - wheelchairs
Triple Query Result: [False]

35:   While you have-
stop words: you, 

Before : you - have- - 
Method1: you - have- - 
Method2: - have- - 

Triple for Query   : you - have- - 
Triple Query Result: [False]

36: some kids who are ambulatory but a lot of the kids who are in wheelchairs their life's vision is from about four feet - about four-foot level
stop words: some, who, are, but, a, of, the, who, are, in, their, is, from, about, four, about, four, 

Before : some kids - be - 
Method1: kids - be - 
Method2: kids - be - 

Triple for Query   : some kids - be - 
Triple Query Result: [False]

37:   Anything above that is outside of their normal element
stop words: above, that, is, of, their, 

Before :   Anything - be - 
Method1:   Anything - be - 
Method2:    Anything - be - 

Triple for Query   :   Anything - be - 
Triple Query Result: [False]

38:   We try to bring those elements to them and we try to do that a

Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

71: When we talk about what's that tree house incorporate it's how do we get kids up so that it's not just the zip line experience it's the whole getting up and into the canopy of the tree to be a part of nature
stop words: we, about, what, that, it, how, do, we, get, up, so, that, it, not, just, the, it, the, whole, up, and, into, the, of, the, to, be, a, part, of, 

Before : it - be - the whole
Method1: it - be - whole
Method2: - be - 

Triple for Query   : it - be - the whole
Triple Query Result: [False]

72:   And what does that look like for a camper um prospectively?  So getting their ideas is important
stop words: what, does, that, for, a, their, is, 

Before : ideas - get - 
Method1: ideas - get - 
Method2: ideas - get - 

Triple for Query   : ideas - get - 
Triple Query Result: [False]

73:   That's how we generate
stop words: how, we, 

Before : we - be - 
Method1: we - be - 
Method2: - be - 

Triple fo

Method2: - within - 

Triple for Query   :  - within - 
Triple Query Result: [False]

108: 
stop words: 

Before :  -  - 
Method1:  -  - 
Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

109: Yeah
stop words: 

Before :  - yeah - 
Method1:  - yeah - 
Method2: - yeah - 

Triple for Query   :  - yeah - 
Triple Query Result: [False]

110: 
stop words: 

Before :  -  - 
Method1:  -  - 
Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

111: Um a- actually Matt if you could just kind of explain to us -
stop words: if, you, could, just, of, to, us, 

Before : you - matt - 
Method1: you - matt - 
Method2: - matt - 

Triple for Query   : you - matt - 
Triple Query Result: [False]

112: exactly what your function is maybe that will help us kind of direct better questions to you
stop words: what, your, is, that, will, us, of, to, you, 

Before : your function - be - what
Method1: function - be - what
Method2: function - be - 

Triple for Query   : 

...

Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

353: One approach to that may be though accommodating able-bodied people um
stop words: to, that, may, be, though, 

Before : One approach - be - people
Method1: One approach - be - people
Method2: One approach - be - people 

Triple for Query   : One approach - be - people
Triple Query Result: [False]

354:   It might be that similar to what you do at a main entrance
stop words: might, be, that, to, what, you, do, at, a, 

Before :   It - be - what
Method1:   It - be - what
Method2:    It - be - 

Triple for Query   :   It - be - what
Triple Query Result: [False]

355:   The ramp is primary but you also have another route that might be steps or or a steeper ramp that's sort of the the express route for accommodating people from two different um paths
stop words: is, but, you, also, have, another, that, might, be, or, or, a, that, of, the, the, for, from, two, 

Before :   The ramp - be - people
Method1:   The ra


390: Well somebody mentioned the lighting in the night at dark
stop words: the, in, the, at, 

Before : somebody - mention - the lighting
Method1: somebody - mention - lighting
Method2: somebody - mention - lighting 

Triple for Query   : somebody - mention - the lighting
Triple Query Result: [False]

391: 
stop words: 

Before :  -  - 
Method1:  -  - 
Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

392: That could that could be possible
stop words: could, that, could, be, 

Before : that - be - 
Method1: that - be - 
Method2: - be - 

Triple for Query   : that - be - 
Triple Query Result: [False]

393:   You'd have a you'd have to have a one -
stop words: have, a, you, have, to, have, a, one, 

Before : you - have - one
Method1: you - have - one
Method2: - have - 

Triple for Query   : you - have - one
Triple Query Result: [False]

394: Video cuts out and comes back in 0:31:06]
stop words: out, and, back, in, 

Before : Video - cut - 
Method1: Video - cut - 


435: Putting in skylights?
stop words: in, 

Before :  - put - skylights
Method1:  - put - skylights
Method2: - put - skylights 

Triple for Query   :  - put - skylights
Triple Query Result: [False]

436: Yeah skylights 
stop words: 

Before :  - yeah - 
Method1:  - yeah - 
Method2: - yeah - 

Triple for Query   :  - yeah - 
Triple Query Result: [False]

437: 
stop words: 

Before :  -  - 
Method1:  -  - 
Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

438: But yeah don't don't - you guys don't have to get locked into the idea that it's a totally roofed-in structure like this
stop words: do, do, you, do, have, to, get, into, the, that, it, a, in, this, 

Before : you guys - have - 
Method1: guys - have - 
Method2: guys - have - 

Triple for Query   : you guys - have - 
Triple Query Result: [False]

439:   Maybe there is a hole where the tree could
stop words: there, is, a, where, the, could, 

Before : tree - be - a hole
Method1: tree - be - hole
Method2: tre


471: So you said you want it to stand like away from like the like anything that's like having to do with playground or you wanted it more like rustic camping?
stop words: you, you, it, to, from, the, anything, that, to, do, with, or, you, it, more, 

Before : you - want - 
Method1: you - want - 
Method2: - want - 

Triple for Query   : you - want - 
Triple Query Result: [False]

472: That's just Daniel's preference you know
stop words: just, you, 

Before : you - be - just Daniel's preference
Method1: you - be - Daniel's preference
Method2: - be - Daniel 's preference 

Triple for Query   : you - be - just Daniel's preference
Triple Query Result: [False]

473: 
stop words: 

Before :  -  - 
Method1:  -  - 
Method2: -  - 

Triple for Query   :  -  - 
Triple Query Result: [False]

474: 'Cause ah one of the things we were talking about is having maybe like since it's gonna be multilevel maybe having like a slide that comes out one side and goes into another side as like getting from one


510: Okay
stop words: 

Before :  - okay - 
Method1:  - okay - 
Method2: - okay - 

Triple for Query   :  - okay - 
Triple Query Result: [False]

511:    
stop words: 

Before :  -     - 
Method1:  -     - 
Method2: -     - 

Triple for Query   :  -     - 
Triple Query Result: [False]

512: It's the prototype for a 
stop words: the, for, a, 

Before : It - be - the prototype
Method1: It - be - prototype
Method2: It - be - prototype 

Triple for Query   : It - be - the prototype
Triple Query Result: [False]

513:   But it - that chair right now is is usable for any of the kids who are here at at camp today
stop words: it, that, now, is, is, for, any, of, the, who, are, here, at, at, 

Before : that chair - be - 
Method1: chair - be - 
Method2: chair - be - 

Triple for Query   : that chair - be - 
Triple Query Result: [False]

514:   It accommodates our kids but it needs a few more adaptations but it's it's 100 percent - I've been on it
stop words: our, but, it, a, few, more, but, it, 

In [49]:
# to remove WARNINGs from Owlready2
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')


In [59]:
# reference: https://pythonhosted.org/Owlready2/world.html
from owlready2 import *
import rdflib

my_world = World()# Owlready2 stores every triples in a ‘World’ object
onto = my_world.get_ontology("dbpedia.owl").load()

graph = my_world.as_rdflib_graph()
print(len(graph))

prefix = """
PREFIX rdf:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbpd:<http://dbpedia.org/ontology/>
"""

'''r = list(graph.query(prefix + """
SELECT ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))'''

'''r = list(graph.query(prefix + """
ASK {
  dbpd:provinceLink rdf:range  dbpd:Province.
}"""))'''

r = list(graph.query(prefix + """
DESCRIBE ?sub WHERE {
  ?sub rdf:range  dbpd:Province.
}"""))

print(r)

* Owlready2 * WARNING: http://dbpedia.org/ontology/senator belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.MemberOfParliament, DUL.sameSettingAs]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/politicGovernmentDepartment belongs to more than one entity types (e.g. Class, Property, Individual): [owl.ObjectProperty, dbpedia.Department, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/productShape belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasQuality]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/latinName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


31050


* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6391Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6393Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/iso6392Code belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.LanguageCode]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/ingredientName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, DUL.hasPart]; I'm trying to fix it...
* Owlready2 * WARNING: http://dbpedia.org/ontology/greekName belongs to more than one entity types (e.g. Class, Property, Individual): [owl.DataProperty, dbpedia.Name]; I'm trying to fix it...


Exception: DESCRIBE not implemented